In [1]:
import pandas as pd
import json,re
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import numpy as np
import matplotlib.pyplot as plt
from pylab import rcParams
import folium,warnings
from folium.plugins import HeatMap


rcParams["figure.figsize"] = 20,9
warnings.filterwarnings("ignore")



analyzer = SentimentIntensityAnalyzer()

In [2]:

coordinate,text=[],[]
locationList = [[-44.11,111.87],[-10.64,156.79]] #bottom-left，top-right

def appendLocation(coordinate, file):
    with open(file, 'r', encoding="utf-8") as f:
        for line in f:
            try:
                tempPolygon = json.loads(line)['place']['bounding_box']['coordinates'][0]
                
                #'''
                centreLatitude = round((tempPolygon[0][1] + tempPolygon[1][1])/2,3)                         # round to 3 significant digits
                centreLongitude = round((tempPolygon[1][0] + tempPolygon[2][0])/2,3)                        # round to 3 significant digits
                if (centreLatitude <= locationList[1][0] and centreLatitude >= locationList[0][0] and
                    centreLongitude <= locationList[1][1] and centreLongitude >= locationList[0][1]):      #check if the point is within bounding box
                    text = json.loads(line)['text']
                    text = re.sub("RT |(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",text)
                    #analysis = TextBlob(text).sentiment.polarity         #original sentiment polarity
                    score = analyzer.polarity_scores(text)
                    if score['compound'] > 0:
                        color = 'red'
                    elif score['compound'] == 0:
                        color = 'yellow'
                    else:
                        color = 'blue'
                    coordinate.append([centreLatitude,centreLongitude,color])
                #'''
            except:
                continue
appendLocation(coordinate,'tweets.json')
appendLocation(coordinate,'tweets2.json')
appendLocation(coordinate,'tweets3.json')
appendLocation(coordinate,'tweets4.json')
appendLocation(coordinate,'tweets5.json')

In [3]:
df = pd.DataFrame(coordinate)
df.columns =['lat','long','compound']
clean = df[['lat','long','compound']].copy()
clean

,lat,long,compound
0,-18.188,125.573,yellow
1,-33.848,150.932,yellow
2,-27.656,152.909,yellow
3,-33.848,150.932,red
4,-32.150,115.928,yellow
...,...,...,...
1391,-37.973,145.053,blue
1392,-32.150,115.928,yellow
1393,-33.848,150.932,yellow
1394,-37.973,145.053,yellow


In [4]:
# 用csv读column name前面多一个空格, 省去了rename步骤
school = pd.read_csv('2016AustraliaSchoolProfile.csv')
print(len(school))    #total entry of schools
school.drop(school[school[' total_enrolments'] < 200].index,inplace=True)
school.dropna(subset=[' total_enrolments'], how='all', inplace=True)       #drop NaN value
df2 = school[[' latitude',' longitude']].copy()
print(len(df2))       #use schools with >200 enrolemnts
df2

9557
5950


,latitude,longitude
1,-33.271886,151.426188
2,-33.286947,149.079358
3,-36.367444,145.415292
5,-27.603608,152.846753
6,-21.420391,149.213682
...,...,...
9524,-27.093600,152.915800
9532,-31.626991,115.702657
9533,-31.745353,116.008624
9534,-32.124810,115.936929


In [5]:
housing = pd.read_csv('VIC2019HousePrice.csv')
len(housing[' SA22016Name'].unique())  #total numbers of suburbs in VIC

456

In [6]:
map_1=folium.Map(location=[-37.8136, 144.9631], 
                 tiles = "OpenStreetMap",
                zoom_start=5)

for index, row in clean.iterrows():
    folium.CircleMarker(location=[row['lat'],row['long']], radius=5, color=row['compound'], fill_color=row['compound']).add_to(map_1)

#HeatMap(data=clean, radius=20).add_to(map_1)

map_1       #red-> positive; yellow-> neutral; blue-> negative

In [7]:
# transform suburbs to longitude,latitude or vice versa via geocoder
from geopy.geocoders import Baidu, Bing, GoogleV3, DataBC, Nominatim
#geolocator = GoogleV3('') #API key to be continued.

#location = geolocator.geocode("Collingwood,VIC")
#print(location.address)
#print((location.latitude, location.longitude))